# Zipf Law Revisited

In the earlier notebook we saw that the frequencies observed roughly obeyed Zipf's law...

However the fit tailed off towards the less frequent words, now we see that that is to be expected, and how to fix it.

## Biased estimators

What is the best estimate for a word's aymptotic frequency over a large enough sample size? The estimator we might reach for is one based on a sample, as we used in the previous notebook:

$$

\hat{f}(w_i) = \frac{C(w_i)}{|T|}

$$

Where $C$ is the count function, and $|T|$ the total size of the sample text.

In the absence of any underlying model for word frequencies, this does seem fair. However we know that all words do occur with non-zero frequency (otherwise why would they be in a dictionary?). Thus the estimator for an un-observed word, $\bar{w}_i$, $\hat{f}(\bar{w}_i) = 0$ must be biased, the law of total probability then means that our remaining estimators are also biased - or at least a subset of them are.

## Survivorship bias

Intuitively this is due to a survivorship bias - if the expected number of occurences of a word in the sample text is, say $0.5$, then due to the discrete nature of the count function $C : \{ w_i \in W \} \to \mathbb{N}$, we must either under or overestimate it's frequency - and all the observed words like this will be overestimated, conversely for the un-observed words.

## Key Ideas

So what can be do about this? It seems we need to adjust our estimates, especially for words only observed a handful of times. That does align with the break-away from the theory we saw in the previous notebook's graphs. To solve this problem we rely on the following key observations:

1. The number of words we **didn't** observe is also a valuable piece of information
2. Above a certain number of observations, a word's frequency estimate is essential **unbiased**
3. We have a well backed theory for the qualititative relationship frequencies obey, as a function of word rank
4. Using our sample statistics to estimate orderings is **not** subject to bias
5. Estimating rank and ordering are not the same thing

We observe a subtle distinction here, that is worth repeating. Whilst we cannot use $\hat{f}$ to estimate the frequencies we require, we can use it as a unbiased estimate for the ordering of the words by frequency. Then we need to consider the even subtler distinction between rank and order. We now discuss each point.


### Technical notes

"bias" according to the traditional definition is being used incorrectly here. In fact $\hat{f}$ **is** unbiased as our sample size grows to infinity. However Zipf scores are used to discuss occurences of rare words in the region of 10 per billion - talking about asymptotics would require data volumes well beyond the current system can produce.

To formalise this rigorously we would need to consider a dictionary that grows with the sample size, so we always anticipate unseen words. However that means we have a changing data distribution, and would require much more advanced tools - and tangential considerations not helpful for actually solving our present issue.

### 1. Unobserved words

let $T$ be a text distributed according the global Zipf-Mandelbrot law. We let

$$ N = |T| $$ 

be the number of words, (counting repetitions) present in $T$, and 

$$M = | \text{dictionary} |$$

the number of *unique* words we drew from in sampling $T$. We are interested in the following quantity:

$$ U = | \{ w_i | w_i \notin T \} |$$

The total number of unseen words. Given a text, we can trivially exactly compute this - however there is also the following estimator for a given $M$:

$$ \hat{U} = | \text{dictionary} | - \mathbb{E}[\text{unique}] = \mathbb{E} \left[ M - \sum_{w_i} I_i \right]$$

Where $I_i$ is the indicator variable for the event "word $i$ present in $T$", these are not necessarily indepedant random variables, but the linearity of expectation applies nontheless:

$$ \hat{U} = M - \sum_{w_i} \mathbb{E}(I_i) = M - \sum_{w_i} \left(  1 - (1-f(w_i))^M \right) = \sum_{w_i} (1-f(w_i))^M $$

I.e. using the probability that $I_i = 0$, the word was never observed, to estimate the expectation. Here $f$ is the true frequency per word, thus equivalent to the probability any given sampled word in the text is $w_i$.


Taking step back, we see we have derived a formula for an quantity we already know, $U$ ...however the right hand side is interesting, because while know the qualitative shape of $f$, the parameters are unknowns to be fitted - and now we have an extra piece of information to help that, from words that we have never even seen!


### 2. When do we get bias?

We know bias arises from survivorship of a word $w_i$ observed in the text; we do not have any information about the counterfactual - that the text did not contain $w_i$

So how likely was that? lets do a quick case study with a word, that asymptotically had $0.5$ expected occurences in $T$, but we observed it $n$ times:

#### $n = 1$

This case is likely:

$$ \mathbb{P}(n = 1) = {N \choose 1} \cdot \left( \frac{0.5}{N}\right)^1 \cdot \left( 1 - \frac{0.5}{N}\right)^{N-1} \geq N \cdot \frac{0.5}{N} \cdot \left( 1 - (N-1)\frac{0.5}{N}\right)$$

By the Binomial inequality, thus:

$$
 \mathbb{P}(n = 1) \geq 0.25
$$

Definitely worth considering

#### $n = 10$

$$ \mathbb{P}(n = 10) = {N \choose 10} \cdot \left( \frac{0.5}{N}\right)^{10} \cdot \left( 1 - \frac{0.5}{N}\right)^{N-10} \leq {N \choose 10} \cdot \left( \frac{0.5}{N}\right)^{10} $$

Thus 
$$
\mathbb{P}(n = 10) \leq N \cdot (N-1) \dots (N-10) \cdot \left( \frac{0.5}{N}\right)^{10} \leq N^{10} \cdot \left( \frac{0.5}{N}\right)^{10}
$$

$$
\mathbb{P}(n = 10) \leq 0.5^{10}
$$

A rough upper bound, already severly unlikely, a full analysis of the likelihood that $f(w_i) = 0.5$ given 10 occurences would require some Bayesian reasoning, but regardless it is clear that the effects of survivorship bias are minimal given sufficient observations of a word.

Intuitively, for a word with 10 observations, this is just as likely to be an undersample as an oversample - whereas with 1 observation, we should be considering survivorship bias.


Going the other way, considering the a word with 10 expected occurences, and its survival likelihood. For large $ M $, the Binomial distribution approximates a Poisson distribution with mean $\lambda $, leading to:

$$
\mathbb{P}(n = 0) \approx e^{-\lambda}.
$$

Substituting $\lambda = 10 $, we obtain:

$$
\mathbb{P}(n = 0) \approx e^{-10}.
$$

Numerically,

$$
e^{-10} \approx 4.54 \times 10^{-5}.
$$

Thus, the probability that a word with an expected frequency of 10 in a text of size $ M $ $never appears is approximately $4.54 \times 10^{-5}$ which is very small. Thus the event we didn't observe the word can safely be ignored, and adds minimal bias to our estimate.

To conclude - our estimates for frequencies are decent once we have enough data, this will prove useful in the next point.

### 3. A qualitative model for frequency

Previously we introduced the Zipf-Mandelbrot model:

$$
\text{Frequency} = \frac{c}{(\text{rank} + b)^a}
$$

With fittable parameters $a,b,c$ - we assume this to be true. Then given 2., we can be happy with a fit based on data filtering out the infrequent obervations.

The problem is that now we have a large extrapolation... if only we had a way to tune the long range effects of $a$, based on unbiased data. Well 1. gives us precisely that capability.


### 4. Estimating order

Our recorded frequencies give us a natural partial ordering, and we observe that changing this would not make any sense. Even if we know that an observed word probably shouldn't have been observed - that doesn't mean we should order an unobserved work above it.

As a discrete object (the ordering could be a matrix of 1s, 0s, 0.5s) we have an unbiased estimate of the true ordering (a matrix of 0s and 1s say)

### 5. Why Rank matters

Every order gives a natural function to the integers to yield the rank of a word, ready to be plugged into our model from 3.

We might be tempted to conclude from 4. that we have an unbiased estimator for this... however following this reasoning we arrive at contradictions. (ignoring the fact that the rank function is not even neccessarily well defined)

A key observation is that an estimator of rank merely maps:

$$
\hat{r} : \text{dictionary} \to \mathbb{R}
$$

And that we can then extract a partial/total ordering naturally, the Zipf-Mandelbrot model conflates these two because strictly* monotonic rank functions form equivalence classes with natural representatives: the pullback from the integer valued orders' indices (*i.e.when we have a total ordering)

But with a partial ordering, many different rank functions can yield the same partial ordering, and it is not obviouse where to set the estimator value for, say, all the words we observed exactly 3 times.

We assume that this rank is constant for all these words - which is necessary anyway viewing the $\hat{r}$ as defining a partial order. 

We can then use 4. and consider the *order-range* we are covering, since our ordering is valid. Using our Zipf-Mandelbrot model, we can compute the probability mass present in this range, then divide it equally between the words, then work backwards to get our $\hat{r}$ estimate.

This approach ties together the ideas we've considered, and should lead to a much better fit, when we only have a limited sample size.